In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split # (保留，但仅用于示例)
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, ConfusionMatrixDisplay, RocCurveDisplay
from scipy.stats import ks_2samp

# --- 新增的库 ---
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.exceptions import NotFittedError

# --- 环境设置 ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
# plt.rcParams['font.sans-serif'] = ['SimHei']  # (根据您的环境启用)
# plt.rcParams['axes.unicode_minus'] = False # (根据您的环境启用)

print("--- 库导入和环境设置完成 ---")


# =============================================================================
# 2. 数据加载与预处理 (基本同您版本, 关键修改点已标注)
# =============================================================================
print("\n--- [阶段一] 开始数据加载与预处理 ---")

try:
    file_path = 'accepted_2007_to_2018q4.csv'
    df = pd.read_csv(file_path, low_memory=False)
    print(f"数据加载成功，原始数据形状: {df.shape}")

    df['issue_d'] = pd.to_datetime(df['issue_d'], errors='coerce')

    # --- 【修改】时间范围筛选 ---
    # 筛选2007-2014年数据，用于后续的OOT切分
    start_date = pd.to_datetime('2007-01-01')
    end_date = pd.to_datetime('2014-12-31')
    df_filtered = df[(df['issue_d'] >= start_date) & (df['issue_d'] <= end_date)].copy()
    print(f"筛选2007-2014年数据后，形状为: {df_filtered.shape}")

    # --- 特征剔除 (同您版本) ---
    cols_to_drop = [
        'id', 'member_id', 'url', 'desc', 'title', 'emp_title', 'pymnt_plan', 'out_prncp',
        'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
        'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
        'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'acc_now_delinq',
        'chargeoff_within_12_mths', 'delinq_amnt', 'mths_since_last_delinq',
        'mths_since_last_record', 'mths_since_last_major_derog', 'hardship_flag',
        'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term',
        'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date',
        'hardship_length', 'hardship_dpd', 'hardship_loan_status',
        'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
        'hardship_last_payment_amount', 'debt_settlement_flag', 'debt_settlement_flag_date',
        'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage',
        'settlement_term', 'funded_amnt', 'funded_amnt_inv', 'initial_list_status',
        'verification_status_joint'
    ]
    existing_cols_to_drop = [col for col in cols_to_drop if col in df_filtered.columns]
    df_cleaned = df_filtered.drop(columns=existing_cols_to_drop, errors='ignore')
    print(f"剔除贷后及无关特征后，形状为: {df_cleaned.shape}")

    # --- 目标变量Y的定义 (同您版本) ---
    good_status = ['Fully Paid']
    bad_status = ['Charged Off', 'Default']
    df_model_data = df_cleaned[df_cleaned['loan_status'].isin(good_status + bad_status)].copy()
    df_model_data['Y'] = df_model_data['loan_status'].apply(lambda x: 0 if x in good_status else 1)
    df_model_data = df_model_data.drop(columns=['loan_status'])
    print(f"定义Y并筛选后，数据集形状: {df_model_data.shape}")

    # --- 【删除】数据采样 ---
    # (已删除您原有的随机采样步骤。我们将使用2007-2014全部数据进行OOT划分)
    final_df = df_model_data.copy()
    print(f"使用全部 {len(final_df)} 条数据进行OOT划分。")


    # --- 特征工程与格式转换 (同您版本) ---
    print("开始进行特征工程与格式转换...")
    if 'term' in final_df.columns: final_df['term'] = final_df['term'].str.extract('(\d+)').astype(float)
    if 'int_rate' in final_df.columns: final_df['int_rate'] = final_df['int_rate'].astype(float) / 100.0
    if 'revol_util' in final_df.columns: final_df['revol_util'] = final_df['revol_util'].astype(float) / 100.0
    emp_map = {'< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4, '5 years': 5, '6 years': 6,
               '7 years': 7, '8 years': 8, '9 years': 9, '10+ years': 10}
    if 'emp_length' in final_df.columns: final_df['emp_length'] = final_df['emp_length'].map(emp_map)
    if 'earliest_cr_line' in final_df.columns: final_df['earliest_cr_line'] = pd.to_datetime(
        final_df['earliest_cr_line'], errors='coerce')
    if 'issue_d' in final_df.columns: final_df['issue_d'] = pd.to_datetime(final_df['issue_d'], errors='coerce')
    if 'earliest_cr_line' in final_df.columns and 'issue_d' in final_df.columns:
        final_df['credit_history_months'] = ((final_df['issue_d'] - final_df['earliest_cr_line']).dt.days) / 30.0

    # --- 【修改】删除特征列表 ---
    # 我们必须保留 'issue_d' 用于OOT切分，在切分后再删除它
    cols_to_remove_after_processing = ['earliest_cr_line', 'zip_code', 'addr_state', 'sub_grade',
                                       'emp_title']
    final_df = final_df.drop(columns=[col for col in cols_to_remove_after_processing if col in final_df.columns])

    # --- 缺失值处理 (同您版本) ---
    print("开始处理缺失值...")
    cols_to_fill_999 = ['mths_since_rcnt_il', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq',
                        'mths_since_recent_inq', 'mo_sin_rcnt_rev_tl_op', 'mths_since_recent_bc', 'mo_sin_rcnt_tl']
    cols_to_fill_0 = ['il_util', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'open_acc_6m', 'open_act_il',
                      'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'total_bal_il',
                      'emp_length', 'pub_rec_bankruptcies', 'collections_12_mths_ex_med', 'mo_sin_old_il_acct',
                      'num_tl_120dp_2m', 'avg_cur_bal', 'mo_sin_old_rev_tl_op', 'num_actv_rev_tl', 'num_il_tl',
                      'num_op_rev_tl', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths',
                      'mort_acc', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_bc_sats', 'num_bc_tl',
                      'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
                      'num_tl_op_past_12m', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
                      'total_il_high_credit_limit']
    cols_to_fill_median = ['dti', 'revol_util', 'credit_history_months', 'last_fico_range_high', 'last_fico_range_low',
                           'pct_tl_nvr_dlq', 'bc_open_to_buy', 'bc_util', 'percent_bc_gt_75']
    ALL_COLS_TO_KEEP = cols_to_fill_999 + cols_to_fill_0 + cols_to_fill_median
    missing_rates = final_df.isnull().sum() / len(final_df)
    high_missing_cols = missing_rates[missing_rates > 0.4].index.tolist()
    cols_to_actually_drop = [col for col in high_missing_cols if col not in ALL_COLS_TO_KEEP]
    if cols_to_actually_drop: final_df = final_df.drop(columns=cols_to_actually_drop)
    for col in cols_to_fill_999:
        if col in final_df.columns: final_df[col].fillna(999, inplace=True)
    for col in cols_to_fill_0:
        if col in final_df.columns: final_df[col].fillna(0, inplace=True)
    for col in cols_to_fill_median:
        if col in final_df.columns and final_df[col].isnull().any():
            median_val = final_df[col].median()
            final_df[col].fillna(median_val, inplace=True)
    print("缺失值填充完成。")

    # --- 异常值处理 (同您版本) ---
    print("开始处理异常值...")
    cols_for_strict_cap = ['annual_inc']
    cols_for_standard_cap = ['dti', 'revol_bal', 'tot_cur_bal', 'total_rev_hi_lim',
                             'tot_hi_cred_lim', 'total_bal_ex_mort', 'avg_cur_bal']
    for col in cols_for_strict_cap:
        if col in final_df.columns:
            upper_bound = final_df[col].quantile(0.995)
            final_df[col] = np.clip(final_df[col], a_min=None, a_max=upper_bound)
    for col in cols_for_standard_cap:
        if col in final_df.columns:
            upper_bound = final_df[col].quantile(0.99)
            final_df[col] = np.clip(final_df[col], a_min=None, a_max=upper_bound)
    all_numeric_cols = final_df.select_dtypes(include=np.number).columns.tolist()
    if 'Y' in all_numeric_cols: all_numeric_cols.remove('Y')
    if 'issue_d' in all_numeric_cols: all_numeric_cols.remove('issue_d') # 排除 'issue_d'
    processed_cols = cols_for_strict_cap + cols_for_standard_cap
    remaining_numeric_cols = [col for col in all_numeric_cols if col not in processed_cols]
    for col in remaining_numeric_cols:
        lower_bound = final_df[col].quantile(0.01)
        upper_bound = final_df[col].quantile(0.99)
        final_df[col] = np.clip(final_df[col], lower_bound, upper_bound)
    print("异常值处理完成。")

    # --- 特征精简 (同您版本) ---
    print(f"精简前原始形状: {final_df.shape}")
    cols_to_drop_manually = [
        'collections_12_mths_ex_med', 'policy_code', 'open_acc_6m', 'open_act_il',
        'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
        'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi',
        'total_cu_tl', 'inq_last_12m', 'num_tl_120dpd_2m', 'num_tl_30dpd',
        'fico_range_low', 'grade', 'last_fico_range_high', 'last_fico_range_low'
    ]
    existing_cols_to_drop = [col for col in cols_to_drop_manually if col in final_df.columns]
    final_df_filtered = final_df.drop(columns=existing_cols_to_drop)
    print(f"手动移除了 {len(existing_cols_to_drop)} 个特征。")
    print(f"精简后形状 (准备OOT切分): {final_df_filtered.shape}")
# =========================================================================
    # --- 【新增V4】 核心特征筛选 (Less is More) ---
    # =========================================================================
    print("\n--- [阶段一-B] 执行 V4 核心特征筛选 ---")
    print("剔除所有高PSI风险的复杂征信局特征...")

    # 我们只保留最核心、最稳定的 申请/信用 特征
    CORE_FEATURES_TO_KEEP = [
        # 目标变量
        'Y',

        # OOT 切分键
        'issue_d',

        # 申请信息 (稳定)
        'loan_amnt',             # 贷款金额
        'term',                  # 期限
        'int_rate',              # 利率 (关键风险定价)
        'installment',           # 月供
        'purpose',               # 用途
        'home_ownership',        # 住房情况
        'emp_length',            # 工作年限

        # 核心信用 (稳定)
        'fico_range_high',       # FICO 分
        'annual_inc',            # 年收入
        'dti',                   # DTI
        'revol_util',            # 循环额度使用率
        'verification_status',   # 验证状态

        # 保留几个最简单的历史变量
        'delinq_2yrs',           # 2年逾期
        'pub_rec_bankruptcies',  # 破产记录
        'credit_history_months'  # 信用历史
    ]

    # 检查 final_df_filtered 中有哪些我们想保留的列
    existing_core_cols = [col for col in CORE_FEATURES_TO_KEEP if col in final_df_filtered.columns]

    print(f"原始特征数: {len(final_df_filtered.columns)}")
    final_df_filtered = final_df_filtered[existing_core_cols]
    print(f"核心模型特征数 (V4): {len(final_df_filtered.columns)}")

    print("--- [阶段一-B] V4 核心特征筛选完成 ---")

    print("--- [阶段一] 数据预处理全部完成 ---")

except FileNotFoundError:
# ( ... 您的 except 块保持不变 ...)
    print("--- [阶段一] 数据预处理全部完成 ---")

except FileNotFoundError:
    print("\n错误: 'accepted_2007_to_2018q4.csv' 未找到。请确保文件在脚本所在的目录中。")
    # 退出程序或在Jupyter中停止
    # exit()

--- 库导入和环境设置完成 ---

--- [阶段一] 开始数据加载与预处理 ---
数据加载成功，原始数据形状: (2260701, 151)
筛选2007-2014年数据后，形状为: (466345, 151)
剔除贷后及无关特征后，形状为: (466345, 99)
定义Y并筛选后，数据集形状: (451060, 99)
使用全部 451060 条数据进行OOT划分。
开始进行特征工程与格式转换...
开始处理缺失值...
缺失值填充完成。
开始处理异常值...
异常值处理完成。
精简前原始形状: (451060, 81)
手动移除了 22 个特征。
精简后形状 (准备OOT切分): (451060, 59)

--- [阶段一-B] 执行 V4 核心特征筛选 ---
剔除所有高PSI风险的复杂征信局特征...
原始特征数: 59
核心模型特征数 (V4): 17
--- [阶段一-B] V4 核心特征筛选完成 ---
--- [阶段一] 数据预处理全部完成 ---


In [58]:
# =============================================================================
# 3. 【新增】工业级OOT (Out-of-Time) 数据集切分 (V2 - 滚动窗口)
# =============================================================================
print("\n--- [阶段二] 开始OOT (Out-of-Time) 切分 ---")
print("使用滚动时间窗口 (V2): 训练数据更接近测试数据，以对抗客群偏移")

# --- 新的、更近的时间窗口 ---
# 1. 训练集 (Train): 2012-01-01 -> 2013-06-30
train_df = final_df_filtered[
    (final_df_filtered['issue_d'] >= '2012-01-01') &
    (final_df_filtered['issue_d'] < '2013-07-01')
].copy()

# 2. 验证集 (Validation): 2013-07-01 -> 2013-12-31 (用于调参和早停)
val_df = final_df_filtered[
    (final_df_filtered['issue_d'] >= '2013-07-01') &
    (final_df_filtered['issue_d'] < '2014-01-01')
].copy()

# 3. 测试集 (Test/OOT): 2014-01-01 -> 2014-12-31
test_df = final_df_filtered[
    (final_df_filtered['issue_d'] >= '2014-01-01') &
    (final_df_filtered['issue_d'] <= '2014-12-31') # 确保在2014年内
].copy()

# 4. 分离 X 和 y
y_train = train_df['Y']
X_train = train_df.drop(columns=['Y', 'issue_d']) # 切分后删除 'issue_d'

y_val = val_df['Y']
X_val = val_df.drop(columns=['Y', 'issue_d'])

y_test = test_df['Y']
X_test = test_df.drop(columns=['Y', 'issue_d'])

print(f"训练集 (2012-2013H1): X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"验证集 (2013H2):     X_val:   {X_val.shape}, y_val:   {y_val.shape}")
print(f"测试集 (2014-OOT):   X_test:  {X_test.shape}, y_test:  {y_test.shape}")

if X_train.empty or X_val.empty or X_test.empty:
    print("\n*** 警告: 数据集为空！请检查 [阶段一] 的日期筛选和 [阶段二] 的切分日期是否正确！ ***")


--- [阶段二] 开始OOT (Out-of-Time) 切分 ---
使用滚动时间窗口 (V2): 训练数据更接近测试数据，以对抗客群偏移
训练集 (2012-2013H1): X_train: (106741, 15), y_train: (106741,)
验证集 (2013H2):     X_val:   (81430, 15), y_val:   (81430,)
测试集 (2014-OOT):   X_test:  (223103, 15), y_test:  (223103,)


In [59]:
# =============================================================================
# 3. 【新增V5】特征工程 (基于稳定的核心组件)
# =============================================================================
print("\n--- [阶段三-A] 开始特征工程 (基于稳定的核心组件) ---")

def create_stable_ratio_features(df):
    """
    在DataFrame上创建新的、基于稳定组件的比率特征
    """
    df_new = df.copy()

    # 使用 .loc 避免 SettingWithCopyWarning

    # 1. 收入相关比率 (组件: loan_amnt, installment, annual_inc 都是核心特征)
    #    (我们在[阶段一]的预处理中已经处理了 annual_inc 的极端值)
    df_new['annual_inc_safe'] = df_new['annual_inc'].replace(0, 0.01)
    df_new.loc[:, 'loan_to_income_ratio'] = df_new['loan_amnt'] / df_new['annual_inc_safe']
    df_new.loc[:, 'installment_to_income_ratio'] = df_new['installment'] / (df_new['annual_inc_safe'] / 12)

    # 2. 负债 vs 收入 (组件: dti, annual_inc 都是核心特征)
    #    DTI = (月负债 / 月收入). 我们反推 月负债
    df_new.loc[:, 'monthly_debt'] = (df_new['dti'] * (df_new['annual_inc_safe'] / 12))

    # 3. 信用历史 vs 近期活动 (组件: delinq_2yrs, credit_history_months)
    df_new['credit_history_safe'] = df_new['credit_history_months'].replace(0, 0.01)
    df_new.loc[:, 'delinq_to_history_ratio'] = df_new['delinq_2yrs'] / df_new['credit_history_safe']

    # 4. FICO vs DTI 交叉 (组件: fico_range_high, dti)
    df_new.loc[:, 'fico_x_dti'] = df_new['fico_range_high'] * df_new['dti']

    # 删除用于计算的临时 'safe' 列
    df_new = df_new.drop(columns=['annual_inc_safe', 'credit_history_safe'])

    return df_new

# 在所有数据集上应用特征工程
X_train = create_stable_ratio_features(X_train)
X_val = create_stable_ratio_features(X_val)
X_test = create_stable_ratio_features(X_test)

print(f"特征工程完成。 X_train 新形状: {X_train.shape}")


--- [阶段三-A] 开始特征工程 (基于稳定的核心组件) ---
特征工程完成。 X_train 新形状: (106741, 20)


In [65]:
# =============================================================================
# 4. 【重构V5】特征识别与约束 (核心 + 衍生特征)
# =============================================================================
print("\n--- [阶段三-B] 开始特征识别与约束 (V5 核心+衍生) ---")

# --- 步骤 1: 识别特征类型 ---
CARDINALITY_THRESHOLD = 50
categorical_cols = []
continuous_cols = []

for col in X_train.columns:
    col_dtype = str(X_train[col].dtype)
    col_nunique = X_train[col].nunique()

    if col_dtype in ['object', 'bool', 'category']:
        categorical_cols.append(col)
    elif col_dtype.startswith('int'):
        if col_nunique < CARDINALITY_THRESHOLD:
            categorical_cols.append(col)
        else:
            continuous_cols.append(col)
    elif col_dtype.startswith('float'):
        continuous_cols.append(col)

print(f"识别到 {len(categorical_cols)} 个分类特征。")
print(f"识别到 {len(continuous_cols)} 个连续特征。")

for df in [X_train, X_val, X_test]:
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].astype('category')

print("特征类型转换为 'category' 完成。")

# --- 步骤 2: 定义【完整】的单调性约束 ---
BUSINESS_MONOTONICITY = {
    # 风险负相关 (-1) (越高越好)
    'fico_range_high': -1,   # FICO分 (必须约束)
    'emp_length': -1,        # 工作年限
    'credit_history_months': -1, # 信用历史

    # 风险正相关 (1) (越高越坏)
    'int_rate': 1,         # 利率 (必须约束)
    'term': 1,             # 期限 (必须约束)
    'revol_util': 1,       # 循环额度使用率
    'loan_amnt': 1,        # 贷款金额
    'installment': 1,      # 月供
    'delinq_2yrs': 1,      # 2年逾期
    'pub_rec_bankruptcies': 1, # 破产

    # 【新增】衍生特征的约束
    'loan_to_income_ratio': 1,    # 贷收比 (越高越坏)
    'installment_to_income_ratio': 1, # 月供收入比 (越高越坏)
    'monthly_debt': 1,            # 月负债 (越高越坏)
    'delinq_to_history_ratio': 1, # 逾期/历史比 (越高越坏)
    'fico_x_dti': 1,

    # 【V6 松绑】(设为 0 = 无约束)
    'annual_inc': 0,      # <-- 从 -1 释放为 0
    'dti': 0,              # <-- 从 1 释放为 0

    # 无约束 (0) - 类别特征
    'purpose': 0,
    'verification_status': 0,
    'home_ownership': 0,
}

# --- 自动生成约束列表 ---
monotone_constraints_list = []
for col in X_train.columns:
    constraint = BUSINESS_MONOTONICITY.get(col, 0)
    monotone_constraints_list.append(constraint)

monotone_constraints_tuple = tuple(monotone_constraints_list)

print("基于业务逻辑生成的【V5】单调性约束 (1:增, -1:减, 0:无):")
constraints_series = pd.Series(monotone_constraints_list, index=X_train.columns)
print(constraints_series[constraints_series != 0])

# 最终检查
unconstrained_numeric = [
    col for col in continuous_cols
    if BUSINESS_MONOTONICITY.get(col, 0) == 0 and col in X_train.columns
]
if unconstrained_numeric:
    print(f"\n*** 最终警告: 以下【数值型】特征未被约束，请立刻加入字典: {unconstrained_numeric} ***")
else:
    print("\n(V5) 检查通过：所有数值型特征均已设置约束。")


--- [阶段三-B] 开始特征识别与约束 (V5 核心+衍生) ---
识别到 3 个分类特征。
识别到 17 个连续特征。
特征类型转换为 'category' 完成。
基于业务逻辑生成的【V5】单调性约束 (1:增, -1:减, 0:无):
loan_amnt                      1
term                           1
int_rate                       1
installment                    1
emp_length                    -1
fico_range_high               -1
revol_util                     1
delinq_2yrs                    1
pub_rec_bankruptcies           1
credit_history_months         -1
loan_to_income_ratio           1
installment_to_income_ratio    1
monthly_debt                   1
delinq_to_history_ratio        1
fico_x_dti                     1
dtype: int64

*** 最终警告: 以下【数值型】特征未被约束，请立刻加入字典: ['annual_inc', 'dti'] ***


In [66]:
# =============================================================================
# 5. 【重构V3】模型训练 (Optuna + OOT + 原始特征)
# =============================================================================
print("\n--- [阶段五] 开始模型调优与训练 ---")

# --- 6.1 计算样本权重 (在OOT训练集上) ---
print("计算样本权重 (scale_pos_weight)...")
n_good = (y_train == 0).sum()
n_bad = (y_train == 1).sum()
scale_pos_weight = n_good / n_bad
print(f"训练集中好客户数: {n_good}, 坏客户数: {n_bad}")
print(f"计算得到的 scale_pos_weight: {scale_pos_weight:.4f}")

# --- 6.2 Optuna 目标函数 (使用OOT验证集) ---
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'booster': 'gbtree',
        'n_jobs': -1,
        'random_state': 42,
        'scale_pos_weight': scale_pos_weight,

        # 【关键】约束 和 类别特征
        'monotone_constraints': monotone_constraints_tuple,
        'enable_categorical': True, # <-- 【关键】重新启用

        # Optuna 调优参数
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'min_child_weight': trial.suggest_int('min_child_weight', 20, 100),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 0.1, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9)
    }

    model = xgb.XGBClassifier(
        **params,
        n_estimators=1000,
        early_stopping_rounds=50
    )

    model.fit(
        X_train, y_train, # <-- 使用原始 X_train
        eval_set=[(X_val, y_val)], # <-- 使用原始 X_val
        verbose=False
    )

    return model.best_score


# --- 6.3 运行 Optuna Study ---
print("开始 Optuna 调优... (使用原始特征, 这可能需要几分钟)")
study = optuna.create_study(direction='maximize', study_name='xgb_Acard_Raw_OOT')
study.optimize(objective, n_trials=30, show_progress_bar=True) # (建议增加到 50-100)

print("\n调优完成！")
print(f"最佳试验 (Validation-AUC): {study.best_value:.4f}")
print("找到的最佳超参数:")
print(study.best_params)

# --- 6.4 训练最终模型 ---
print("\n训练最终模型 (使用最佳参数)...")
final_xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'n_jobs': -1,
    'random_state': 42,
    'scale_pos_weight': scale_pos_weight,
    'monotone_constraints': monotone_constraints_tuple,
    'enable_categorical': True,
    'n_estimators': 2000,
    'early_stopping_rounds': 50
}
final_xgb_params.update(study.best_params)

model_xgb_final = xgb.XGBClassifier(**final_xgb_params)

# 在 (训练集) 上训练, 在 (验证集) 上早停
model_xgb_final.fit(
    X_train, y_train, # <-- 原始 X_train
    eval_set=[(X_val, y_val)], # <-- 原始 X_val
    verbose=100
)
print("模型训练完成。")
print(f"模型在早停机制下，最佳迭代次数为: {model_xgb_final.best_iteration}")

[I 2025-10-20 01:29:40,213] A new study created in memory with name: xgb_Acard_Raw_OOT



--- [阶段五] 开始模型调优与训练 ---
计算样本权重 (scale_pos_weight)...
训练集中好客户数: 89702, 坏客户数: 17039
计算得到的 scale_pos_weight: 5.2645
开始 Optuna 调优... (使用原始特征, 这可能需要几分钟)


Best trial: 0. Best value: 0.691921:   3%|▎         | 1/30 [00:05<02:45,  5.71s/it]

[I 2025-10-20 01:29:45,921] Trial 0 finished with value: 0.6919207047610745 and parameters: {'learning_rate': 0.011884761394925792, 'max_depth': 5, 'min_child_weight': 30, 'gamma': 0.3168927610914185, 'lambda': 0.6322260334314332, 'alpha': 0.15736996923646193, 'subsample': 0.7696962201645728, 'colsample_bytree': 0.7171592217273689}. Best is trial 0 with value: 0.6919207047610745.


Best trial: 0. Best value: 0.691921:   7%|▋         | 2/30 [00:10<02:21,  5.05s/it]

...

Best trial: 23. Best value: 0.692071:  97%|█████████▋| 29/30 [01:52<00:04,  4.32s/it]

[I 2025-10-20 01:31:32,762] Trial 28 finished with value: 0.6918083087799818 and parameters: {'learning_rate': 0.018520173863146385, 'max_depth': 4, 'min_child_weight': 50, 'gamma': 0.7296104268215767, 'lambda': 0.3083166548898989, 'alpha': 3.672783812650659, 'subsample': 0.7977833872356648, 'colsample_bytree': 0.6401050536261284}. Best is trial 23 with value: 0.6920712190111711.


Best trial: 23. Best value: 0.692071: 100%|██████████| 30/30 [01:58<00:00,  3.94s/it]

[I 2025-10-20 01:31:38,517] Trial 29 finished with value: 0.6919975627832163 and parameters: {'learning_rate': 0.010574307399516699, 'max_depth': 5, 'min_child_weight': 27, 'gamma': 0.9066893774941525, 'lambda': 4.920730061327121, 'alpha': 0.9410398780669652, 'subsample': 0.7620124600854916, 'colsample_bytree': 0.690172325290005}. Best is trial 23 with value: 0.6920712190111711.

调优完成！
最佳试验 (Validation-AUC): 0.6921
找到的最佳超参数:
{'learning_rate': 0.015518733872983949, 'max_depth': 4, 'min_child_weight': 41, 'gamma': 0.6586612364738023, 'lambda': 1.2306993576347656, 'alpha': 3.571476699907984, 'subsample': 0.7215950782009776, 'colsample_bytree': 0.6484088839582552}

训练最终模型 (使用最佳参数)...
[0]	validation_0-auc:0.67170


[100]	validation_0-auc:0.68910
[200]	validation_0-auc:0.69077
[300]	validation_0-auc:0.69161
[400]	validation_0-auc:0.69201
[483]	validation_0-auc:0.69202
模型训练完成。
模型在早停机制下，最佳迭代次数为: 434


In [69]:
# =============================================================================
# 6. 【重构V3】模型评估 (在OOT测试集上)
# =============================================================================
print("\n--- [阶段六] 模型评估 (在 2014 OOT 测试集上) ---")

# 7.1 获取预测结果 (在 OOT 测试集上)
y_pred_proba_oot = model_xgb_final.predict_proba(X_test)[:, 1] # <-- 原始 X_test

# 7.2 ROC-AUC
auc_score_oot = roc_auc_score(y_test, y_pred_proba_oot)
print(f"OOT 测试集 ROC-AUC 评分: {auc_score_oot:.4f}")

# 7.3 KS 统计量
prob_good_oot = y_pred_proba_oot[y_test == 0]
prob_bad_oot = y_pred_proba_oot[y_test == 1]
ks_stat_oot = ks_2samp(prob_good_oot, prob_bad_oot).statistic
print(f"OOT 测试集 KS 统计量: {ks_stat_oot:.4f}")


--- [阶段六] 模型评估 (在 2014 OOT 测试集上) ---
OOT 测试集 ROC-AUC 评分: 0.7039
OOT 测试集 KS 统计量: 0.2964


In [70]:
# =============================================================================
# 7. 【重构V3】PSI (稳定性监控)
# =============================================================================
print("\n--- [阶段七] 计算 PSI (稳定性监控) ---")

# ( ... 这里的 calculate_psi 函数定义保持不变 ... )
# def calculate_psi(base_array, comparison_array, num_bins=10): ...

# 7.6.1 计算模型分 PSI (训练集 vs OOT测试集)
y_pred_proba_train = model_xgb_final.predict_proba(X_train)[:, 1] # <-- 原始 X_train

score_psi = calculate_psi(y_pred_proba_train, y_pred_proba_oot)
print(f"模型分 PSI (Train vs OOT): {score_psi:.4f}")
# ( ... PSI 结果打印 ... )


# 7.6.2 计算核心特征 PSI
print("\n核心特征 PSI (Train vs OOT):")
feature_psi_results = {}
# 【关键】我们现在只检查被约束的数值特征
constrained_numeric_cols = [
    col for col in continuous_cols
    if BUSINESS_MONOTONICITY.get(col, 0) != 0 and col in X_train.columns
]

for col in constrained_numeric_cols:
    base_feature_data = X_train[col]  # <-- 原始 X_train
    comp_feature_data = X_test[col] # <-- 原始 X_test
    f_psi = calculate_psi(base_feature_data, comp_feature_data)
    feature_psi_results[col] = f_psi

psi_series = pd.Series(feature_psi_results).sort_values(ascending=False)
print(psi_series.head(10)) # 打印PSI最高的10个特征


--- [阶段七] 计算 PSI (稳定性监控) ---
模型分 PSI (Train vs OOT): 0.0067

核心特征 PSI (Train vs OOT):
int_rate                 0.060454
pub_rec_bankruptcies     0.059612
fico_range_high          0.052760
credit_history_months    0.045962
revol_util               0.042764
delinq_2yrs              0.034113
term                     0.022487
emp_length               0.019661
fico_x_dti               0.017379
monthly_debt             0.015193
dtype: float64


In [71]:
# =============================================================================
# 8. 【重构】将概率转换为评分 (Scorecard)
# =============================================================================
print("\n--- [阶段八] 将概率转换为评分 (Scorecard) ---")

# --- 8.1 概率校准 ---
# (您的校准逻辑是正确的，因为 scale_pos_weight 模拟了50/50的平衡)
p_model = 0.5
p_real = y_train.mean() # 使用训练集的真实坏账率
odds_model = p_model / (1 - p_model)
odds_real = p_real / (1 - p_real)
W = odds_real / odds_model
print(f"开始概率校准：模型输出世界(P_model=50.0%) -> 真实世界(P_real={p_real:.1%})")
print(f"修正因子 W = {W:.4f}")

# 【注意】我们只校准OOT测试集的概率
y_prob_uncalibrated_oot = y_pred_proba_oot
y_prob_calibrated_oot = (y_prob_uncalibrated_oot * W) / (1 - y_prob_uncalibrated_oot + y_prob_uncalibrated_oot * W)
print("OOT 测试集概率校准完成。")

# --- 8.2 业务评分 (同您版本) ---
def probability_to_score(prob, base_score=600, base_odds=(1/19), pdo=20):
    factor = pdo / np.log(2)
    offset = base_score - (factor * np.log(base_odds))
    prob = np.clip(prob, 1e-7, 1 - 1e-7)
    odds = prob / (1 - prob)
    score = offset - (factor * np.log(odds))
    return score.astype(int)

# (我们沿用您修改后的参数：Base=650, PDO=50，这组参数在拉伸分数上表现更好)
print(f"\n应用评分卡参数：Base Score = 650, PDO = 50")
scores_oot = probability_to_score(
    y_prob_calibrated_oot,
    base_score=650,
    base_odds=(1/19),
    pdo=50
)

# 8.3 创建结果 DataFrame
df_results_oot = pd.DataFrame({
    'Actual_Y': y_test.values,
    'Prob_Model_Output': y_prob_uncalibrated_oot,
    'Prob_Calibrated': y_prob_calibrated_oot,
    'Score': scores_oot
})

print("概率转换为评分示例 (OOT测试集前10条):")
print(df_results_oot.head(10))

# 8.4 检查好坏客户的分数分布 (OOT测试集)
print("\n分数分布概览 (OOT测试集):")
print(df_results_oot['Score'].describe())

print("\n好客户 (Y=0) 的分数分布 (OOT):")
print(df_results_oot[df_results_oot['Actual_Y'] == 0]['Score'].describe())

print("\n坏客户 (Y=1) 的分数分布 (OOT):")
print(df_results_oot[df_results_oot['Actual_Y'] == 1]['Score'].describe())
print("\n(预期：好客户的平均分应远高于坏客户的平均分)")

print("\n--- XGBoost 工业级流程 (OOT + WOE + PSI) 全部完成 ---")


--- [阶段八] 将概率转换为评分 (Scorecard) ---
开始概率校准：模型输出世界(P_model=50.0%) -> 真实世界(P_real=16.0%)
修正因子 W = 0.1900
OOT 测试集概率校准完成。

应用评分卡参数：Base Score = 650, PDO = 50
概率转换为评分示例 (OOT测试集前10条):
   Actual_Y  Prob_Model_Output  Prob_Calibrated  Score
0         1           0.181975         0.040543   1090
1         0           0.422105         0.121839   1004
2         0           0.458449         0.138527    994
3         1           0.521948         0.171769    975
4         0           0.593920         0.217415    954
5         0           0.444878         0.132116    998
6         1           0.484673         0.151573    986
7         0           0.290286         0.072092   1046
8         1           0.631438         0.245530    943
9         0           0.414848         0.118684   1007

分数分布概览 (OOT测试集):
count    223103.000000
mean        995.523171
std          53.403657
min         826.000000
25%         959.000000
50%         993.000000
75%        1029.000000
max        1231.000000
Name: Score, dt